# Trees and Forests

> Building a simple Decision Tree Classifier!
>
> *Author:* Bjarne C. Hiller

In [2]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin


## Ingredients

- `np.unique` and `np.unique_counts`

## Gini Index

The *Gini index*, named after the mathematician Corrado Gini, is a measure of class impurity, that the Decision Tree tries to minimize.
It represents, of how often a randomly chosen instance would be incorrectly labeled, if labels were decided randomly according to the relative number of instances of the respective class.

Let $Y$ be a multiset of class labels from the underlying class set $C=\{1, ..., c\}$. Then the relative frequencies $p_i$ of each class $i\in C$ is given by:

$$
    p_i = \frac{|\{y \in Y | y=i\}|}{|Y|}
$$

Then, the gini index $I_G$ can be computed as:

$$
    \begin{align*}
    I_G(p)
        &= \sum_{i=1}^c \left( p_i \sum_{j \neq i} p_j \right)\\
        &= \sum_{i=1}^c p_i (1 - p_i)\\
        &= \sum_{i=1}^c (p_i - p_i^2)\\
        &= \sum_{i=1}^c p_i - \sum_{i=1}^c p_i^2\\
        &= 1 - \sum_{i=1}^c p_i^2
    \end{align*}
$$

A gini index of 0 means, that all instances belong to the same class.

In [17]:
def gini_index(labels) -> float:
    """Computes the Gini-Index for a given numpy array of labels."""
    gini = 0
    # TODO: Implement computation of the Gini-Index!
    return gini

In [18]:
def gini_index(labels):
    """Computes the Gini-Index for a given numpy array of labels."""
    _, counts = np.unique_counts(labels)
    p = counts / counts.sum()
    return 1 - np.power(p, 2).sum()


In [19]:
assert gini_index([0,0,0]) == 0
assert gini_index([0,1]) == 0.5

## Decision Trees

In [ ]:
class TreeNode:
    def __init__(self):
        self.left_ = None
        self.right_ = None
        self.feature_ = None
        self.value_ = None
        self.samples_ = None
    
    def apply(self, x):
        return self.value <= x[..., self.feature]
    
    def assign(self, x):
        left = x[..., self.feature_] < self.value_
        return left, ~left
    
    def is_leaf(self):
        return (self.left_ is None) and (self.right_ is None)

    def __repr__(self):
        if self.is_leaf():
            return f"TreeNode({self.samples_})"
        return f"TreeNode({self.samples_}, {self.feature_} < {self.value_:.2f})"


class Tree(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None):
        super().__init__()
        self.max_depth = max_depth
        self.root_ = None
        self.classes_ = None
    
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.root_ = self.grow(X, y, 1)

    def class_counts(self, y):
        """Helper function to receive class counts."""
        bins = np.zeros_like(self.classes_)
        for i, c in enumerate(self.classes_):
            bins[i] = np.sum(y == c)        
        return bins

    
    def grow(self, X, y, depth):
        """Recursively grow a tree."""
        n, p = X.shape
        gini = gini_index(y)
        node = TreeNode()
        node.samples_ = self.class_counts(y)

        # return leaf node
        if gini == 0 or depth == self.max_depth:
            return node

        # test splits
        scores = []
        values = []
        for i in range(p):
            x = np.median(X[:, i])
            scores.append(self.eval_split(X, y, i, x))
            values.append(x)
        
        node.feature_ = np.argmax(scores)
        node.value_ = values[node.feature_]
        
        left, right = node.assign(X)            

        node.left_ = self.grow(X[left], y[left], depth+1)
        node.right_ = self.grow(X[right], y[right], depth+1)
        return node
    
    def eval_split(self, X, y, p, x):
        gini = gini_index(y)
        node = TreeNode()
        node.feature_ = p
        node.value_ = x
        left, right = node.assign(X)
        gini_left = len(y[left]) / len(y) * gini_index(y[left])
        gini_right = len(y[right]) / len(y) * gini_index(y[right])
        return gini - gini_left - gini_right


    def traverse(self, x) -> TreeNode:
        node = self.root_
        while not node.is_leaf():
            left, right = node.assign(x)
            if left.all():
                node = node.left_
            else:
                node = node.right_
        return node

    def predict(self, X):
        predictions = []
        for x in X:
            leaf = self.traverse(x)
            main_class = np.argmax(leaf.samples_)
            y = self.classes_[main_class]
            predictions.append(y)
        return predictions

    def __str__(self):
        text = ""
        nodes = [self.root_]
        while nodes:
            text += "\t".join([str(n) for n in nodes]) + "\n"
            nodes = sum([[n.left_, n.right_] for n in nodes], start=[])
            nodes = [n for n in nodes if n is not None]
        return text


In [59]:
tree = Tree()

In [60]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

iris_ds = load_iris()

X = iris_ds["data"]
y = iris_ds["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [61]:
tree = Tree()
tree.fit(X_train, y_train)

In [62]:
tree.root_

TreeNode([41 42 37], 3 < 1.30)

In [67]:
from sklearn.metrics import accuracy_score

y_pred = tree.predict(X_test)

accuracy_score(y_test, y_pred)

0.9

In [64]:
y

[np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(2),
 np.int64(2),
 np.int64(0),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(2),
 np.int64(2),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1)]

In [65]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, /, start=0)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers

    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [66]:
tree

Tree()